# Generators
Generators are objects that can help with iteration. Rather than constructing an entire list, generators find and return one element at a time, which can often be faster and more space efficient. If we are looking for an element that meets some criteria, we don't have to break and don't have to create every single element. Additionally, since we only create one element at a time, we avoid having to make a massive list that could give us a "MemoryError".

There are two key words used in generators. The first is `yield` which on iteration will return a value and halt at that point. When we need the next value on the next iteration we will continue from that point. The second is `yield from` which, when called on an iterable (list, set, another generator) will continually yield everything from it one at a time before continuing.

In [1]:
def generate_range(n):
    """ 
    Yield the range 0 to n, inclusive
    """
    counter = 0
    while counter <= n:
        yield counter
        counter += 1

In [2]:
for elem in generate_range(10):
    print (elem)

0
1
2
3
4
5
6
7
8
9
10


In [3]:
def generate_range_recursive(n):
    """ 
    Recursively yield the range from 0 to n, inclusive
    """
    if n > 0:
        yield from generate_range_recursive(n-1)
    yield n

In [4]:
for elem in generate_range_recursive(10):
    print (elem)

0
1
2
3
4
5
6
7
8
9
10


## Printing


In [5]:
print (generate_range(10))

<generator object generate_range at 0x000002799F04D480>


In [6]:
print (generate_range(10)[3])

TypeError: 'generator' object is not subscriptable

In [7]:
print (list(generate_range(10)))
print (list(generate_range(10))[3])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
3


## Finding all Subsets

In [2]:
def get_all_subsets(elements, current_index=0, current_subset=None):
    """ 
    Generate a list of all subsets of elements. 
    >>> get_all_subsets([1,2,3])
    [[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]
    """
    results = []
    if current_subset is None:
        current_subset = []
    if current_index < len(elements):
        current_element = elements[current_index]
        # add current element to subset
        results.extend(get_all_subsets(elements, current_index+1, current_subset + [current_element]))
        # skip current element
        results.extend(get_all_subsets(elements, current_index+1, current_subset))
    else:
        results.append(current_subset)
    return results

In [3]:
get_all_subsets([1,2,3])

[[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]

In [4]:
len(get_all_subsets([1,2,3,4,5,6,7]))

128

In [5]:
len(get_all_subsets(list(range(10))))

1024

In [6]:
len(get_all_subsets(list(range(20))))

1048576

As the number of elements grows, the number of elements gets incredibly large. If we make a giant list, it may be too big to fit in memory.

In [7]:
def generate_all_subsets(elements, current_index=0, current_subset=None):
    """ 
    Generate a list of all subsets of elements. 
    >>> list(generate_all_subsets([1,2,3]))
    [[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]
    """
    if current_subset is None:
        current_subset = []
    if current_index < len(elements):
        current_element = elements[current_index]
        # add current element to subset
        yield from generate_all_subsets(elements, current_index+1, current_subset + [current_element])
        # skip current element
        yield from generate_all_subsets(elements, current_index+1, current_subset)
    else:
        yield current_subset

In [9]:
elements = [1,2,3]

In [10]:
for element in generate_all_subsets(elements):
    print ("next:", element)

next: [1, 2, 3]
next: [1, 2]
next: [1, 3]
next: [1]
next: [2, 3]
next: [2]
next: [3]
next: []


In [11]:
for element in get_all_subsets(elements):
    print ("next:", element)

next: [1, 2, 3]
next: [1, 2]
next: [1, 3]
next: [1]
next: [2, 3]
next: [2]
next: [3]
next: []


## Sum of All Paths
For this example, we need a simple tree structure. Here we'll represent a node in a tree as a list with the first element being the node value, and the rest of the list being the children nodes.
That is to say, our tree structure is a simple nested list structure.

<center><img src="tree1.png" alt="tree [13, [7], [8, [99], [16], [42]]]" width=150></center>

In [14]:
tree1 = [13,
            [7],
            [8, 
                [99],
                [16, 
                    [77]
                ],
                [42]
            ]
        ]

We want to find the sum of all paths that start at the root and end at a leaf. Results should be yielded in order from leftmost leaves to rightmost leaves.

The above tree should yield 20, 120, 114, 63 in that order.

In [15]:
def get_sums(tree):
    """ 
    Walk a tree, yielding the sum of each paths starting from the root and ending at a leaf
    >>> list(get_sums([13, [7], [8, [99], [16, [77]], [42]]]))
    [20, 120, 114, 63]
    """
    def helper(tree, cur_sum):
        if len(tree) == 1:
            yield cur_sum + tree[0]
        else:
            for element in tree[1:]:
                yield from helper(element, cur_sum + tree[0])
                
    return helper(tree, 0)

In [16]:
for element in get_sums(tree1):
    print(element)

20
120
114
63


In [17]:
print(list(get_sums(tree1)))

[20, 120, 114, 63]
